In [21]:
!pip install -r requirements.txt
# Gerekli tüm kütüphanelerin tek seferde kurulması
# NOT: Bu komutlar, projenin requirements.txt dosyasında listelenen bağımlılıkları kapsar.
# !pip install -q python-dotenv langchain-google-genai langchain-community langchain-core langchain_text_splitters streamlit


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Cem Yıldız\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
# .env dosyasındaki ortam değişkenlerini yükle
# Güvenlik nedeniyle API Anahtarı doğrudan koda yazılmaz.
import os
from dotenv import load_dotenv

load_dotenv()

try:
    api_key = os.getenv("GEMINI_API_KEY")
    
    if not api_key:
        raise ValueError("Lütfen .env dosyasında 'GEMINI_API_KEY' değerini ayarlayın.")
        
    print("API Anahtarı başarıyla .env dosyasından yüklendi.")
    
except ValueError as e:
    print(f"Hata: {e}")
# API Anahtarının yüklendiğini kontrol et


API Anahtarı başarıyla .env dosyasından yüklendi.


In [13]:
# Gerekli LangChain bileşenlerini içe aktar
from langchain_community.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter 

# 1. Veri setini yükle
# Teknik Anlatım: TextLoader, dosyayı yükler. Encoding='utf-8' parametresi 
# özel Türkçe karakterler nedeniyle ortaya çıkan 'UnicodeDecodeError' hatasını engeller.
loader = TextLoader("yatirim_verileri.txt", encoding="utf-8")
documents = loader.load()

# 2. Metni parçalara ayırma (Text Splitting)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)
# chunk_size=1000: Her parçanın maksimum boyutu.
# chunk_overlap=100: Bağlam kaybını önlemek için parçalar arası örtüşme.

texts = text_splitter.split_documents(documents)

print(f"Yüklenen belge sayısı: {len(documents)}")
print(f"Toplam {len(documents)} belge, {len(texts)} parçaya (chunks) ayrıldı.")
print("\nÖrnek bir parçanın başlangıcı:")
print(texts[0].page_content[:200] + "...")

Yüklenen belge sayısı: 1
Toplam 1 belge, 11 parçaya (chunks) ayrıldı.

Örnek bir parçanın başlangıcı:
=== ALTIN YATIRIMI ===
Altın, tarih boyunca en güvenilir yatırım araçlarından biri olarak kabul edilmiştir. İnsanlık tarihi boyunca para birimi, değer ölçüsü ve zenginlik göstergesi olarak kullanılmış...


In [16]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import os

# 1. API Anahtarını Ortam Değişkenlerinden Çekme
# Bu, daha önceki hücrede (Hücre 1) load_dotenv() ile yüklenmiştir.
api_key = os.getenv("GEMINI_API_KEY")

# 2. Embedding Modelini Tanımlama
# Şimdi anahtarı açıkça 'google_api_key' parametresi ile geçiriyoruz.
embedding_model = GoogleGenerativeAIEmbeddings(
    model="text-embedding-004", 
    google_api_key=api_key # Anahtarı buraya ekliyoruz
) 

# 3. ChromaDB Vektör Veritabanını Oluşturma
print("\nVektör veritabanı oluşturuluyor ve 11 parça vektörleştiriliyor...")

vectorstore = Chroma.from_documents(
    documents=texts,  
    embedding=embedding_model, 
    persist_directory="./chroma_db_yatirim" 
)

# Retriever (Bilgi Çekici) oluşturma
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# Retriever, sorgu geldiğinde vektör veritabanından en alakalı parçaları çekecektir.
# search_kwargs={"k": 3}: Her sorgu için en alakalı 3 (top-k) parçayı çeker.

print("Vektör veritabanı başarıyla oluşturuldu ve diske kaydedildi.")
print(f"Retriever (Bilgi Çekici) hazırlandı. Her sorgu için en alakalı {retriever.search_kwargs['k']} parça çekilecek.")


Vektör veritabanı oluşturuluyor ve 11 parça vektörleştiriliyor...
Vektör veritabanı başarıyla oluşturuldu ve diske kaydedildi.
Retriever (Bilgi Çekici) hazırlandı. Her sorgu için en alakalı 3 parça çekilecek.


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import os

# 1. LLM (Gemini) Modelini Tanımlama
# Teknik Anlatım: gemini-2.5-flash modeli hız ve performans dengesi için seçilmiştir.
# temperature=0.1: Düşük sıcaklık, modelin daha az yaratıcı, daha olgusal (yatırım için kritik) yanıtlar vermesini sağlar.
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0.1
)

# 2. Prompt (İstem) Şablonunu Hazırlama
# Prompt, modele kesin talimatlar verir: SADECE bağlamı kullan ve bilmediğinde reddet.
prompt = ChatPromptTemplate.from_template("""
Sen, Akıllı Yatırım Asistanısın. Görevin, SADECE sağlanan bağlam bilgisine (Context) dayanarak 
kullanıcının yatırım ve finansal konularla ilgili sorularını yanıtlamaktır. 
Eğer bağlamda soruya dair net bir bilgi yoksa, "Üzgünüm, bu konuda elimdeki veri setinde yeterli bilgi bulunmamaktadır." 
diye yanıt ver. Fikir yürütme veya genel bilgi verme.

Bağlam (Context):
{context}

Soru: {question}
""")

# 3. RAG Zincirini Runnable (Çalıştırılabilir) Olarak Kurma (Hata vermez!)
# Bu zincir: Giriş -> Bağlamı Çek (Retriever) -> Prompt'u Hazırla -> LLM'e Gönder -> Yanıtı Çöz (Output Parser)
# RunnableLambda: Retriever'dan gelen liste formatındaki belgeleri (docs), LLM'in isteyeceği tek bir metin dizesine dönüştürür.

def format_docs(docs):
    # Retriever'dan gelen belge listesini (texts), tek bir dizeye dönüştürür
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    # 1. RunnablePassthrough: Girişi alır ve bir sonraki adıma aktarır (question olarak)
    # 2. RunnableLambda: Retriever'ı çalıştırır ve çıktısını format_docs ile işler (context olarak)
    {"context": retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt  # 3. Prompt: context ve question ile doldurulur
    | llm     # 4. LLM: Yanıtı oluşturur
    | StrOutputParser() # 5. Output Parser: Çıktıyı düz metne çevirir
)

print("RAG Zinciri (Runnable Pipeline) başarıyla oluşturuldu. Artık sorgu yapabiliriz.")

# 4. İlk Test Sorgusu
print("-" * 50)
soru = "Portföy çeşitlendirmesi neden önemlidir ve hangi varlıklar önerilir?"
print(f"Kullanıcı Sorusu: {soru}")

# Zinciri çalıştırma
# Runnable zinciri, doğrudan string veya dict alabilir. 'invoke' ile çalıştırıyoruz.
response = rag_chain.invoke(soru)

print("\nAsistanın Yanıtı:")
print(response)


RAG Zinciri (Runnable Pipeline) başarıyla oluşturuldu. Artık sorgu yapabiliriz.
--------------------------------------------------
Kullanıcı Sorusu: Portföy çeşitlendirmesi neden önemlidir ve hangi varlıklar önerilir?

Asistanın Yanıtı:
Portföy çeşitlendirmesi, farklı varlık türlerine yatırım yaparak toplam riski azaltmak anlamına gelir. Bu yöntem, modern portföy teorisinin temelini oluşturur. Örneğin, portföyde hem hisse hem tahvil hem de altın bulunması, tek bir varlığın kötü performans göstermesi durumunda toplam zararı sınırlar.

Önerilen varlıklar yatırımcının risk profiline göre değişir:
*   **Düşük riskli yatırımcılar için:** Mevduat, tahvil, fon ağırlıklı portföy.
*   **Orta riskli yatırımcılar için:** Hisse ve fon karışımı.
*   **Yüksek riskli yatırımcılar için:** Hisse, kripto veya türev ürün ağırlıklı portföy.
